### Librerías

In [1]:
import sys
import cv2
import numpy as np
from matplotlib import pyplot as plt
import imageio
import os
import skvideo.io #para guardar el video



### Funciones para mostrar

In [2]:
def mostrar_imagen_un_canal(image):

  height, width=image.shape
  matrix = np.full((height, width,3), 0, dtype=np.uint8)

  valor_maximo = np.max(image)

  for i in range(height):
      for j in range(width):
          matrix[i][j][0]=image[i][j]/valor_maximo*255
          matrix[i][j][1]=image[i][j]/valor_maximo*255
          matrix[i][j][2]=image[i][j]/valor_maximo*255

  #plt.imshow(matrix)
  return matrix

def mostrar_imagen_tres_canales(imagen):
  height, width,_=imagen.shape
  matrix = np.full((height, width,3), 0, dtype=np.uint8)


  valor_maximo0 = np.max(imagen[:, :, 0])
  valor_maximo1 = np.max(imagen[:, :, 1])
  valor_maximo2 = np.max(imagen[:, :, 2])


  for i in range(height):
      for j in range(width):
          matrix[i][j][0]=imagen[i][j]/valor_maximo0*255
          matrix[i][j][1]=imagen[i][j]/valor_maximo1*255
          matrix[i][j][2]=imagen[i][j]/valor_maximo2*255
  plt.imshow(matrix)



### Lectura de video

In [4]:

ruta_video='D:/Entrada.avi' #ruta del video

cap = cv2.VideoCapture(ruta_video) #Se lee el video

#cap = cv2.VideoCapture("video1_prueba2.mp4") #Se lee el video
frames=[]
cont=0
while True: #guarda el video en un arreglo de frames
    ret,frame=cap.read()
    if(ret==True):
        frames.append(frame)
    else:
        break
width = int(cap.get(3))  # Ancho del video
height = int(cap.get(4))  # Altura del

#frame_prueba=cv2.cvtColor(frames[0], cv2.COLOR_BGR2RGB)#frame usado de prueba para la detección de bordes

#plt.imshow(frame_prueba)
len(frames)


550

### Preprocesamiento

In [5]:
qcd = cv2.QRCodeDetector()
i=0
retval, decoded_info, points, straight_qrcode = qcd.detectAndDecodeMulti(frames[i])

while(retval==False):
    i+=1
    retval, decoded_info, points, straight_qrcode = qcd.detectAndDecodeMulti(frames[i])


        

num_frame_fondo=i-20
num_frame_inicio=i+80


In [9]:


fondo=frames[num_frame_fondo]

# Obtener el ancho y alto del video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))



salida=skvideo.io.FFmpegWriter('D:/resultado.avi') #agregar ruta de salida

for frame in range(num_frame_inicio,len(frames)):

    
    raton=original=cv2.cvtColor(frames[frame], cv2.COLOR_BGR2RGB) #guarda el frame que será procesado


    #hace más chicas las imágenes del fondo y el frame a procesar

    fondo=cv2.resize(fondo,(720,480),interpolation = cv2.INTER_AREA)
    raton=cv2.resize(raton,(720,480),interpolation = cv2.INTER_AREA)

    #Promedio -- suavizado
    fondo=cv2.boxFilter(fondo,ddepth=-1,ksize=(5,5),normalize=True)
    raton=cv2.boxFilter(raton,ddepth=-1,ksize=(5,5),normalize=True)

    height, width,_=fondo.shape
    matrix = np.full((height, width), 0, dtype=np.uint8)
    #Resta en el canal rojo
    for i in range(height):
        for j in range(width):
            matrix[i][j]=np.abs(np.subtract(raton[i][j][0],fondo[i][j][0]))


    imagen=mostrar_imagen_un_canal(matrix)


    #filtro morfológico
    kernelSize=(9,9)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
    img=cv2.morphologyEx(imagen, cv2.MORPH_OPEN, kernel)
    #img=cv2.morphologyEx(raton, cv2.MORPH_OPEN, kernel)

    #Agrandar la imagen
    img=cv2.resize(img,(1920,1080),interpolation = cv2.INTER_AREA)

    #Umbralización 50%

    height, width,_=img.shape

    for i in range(height):
        for j in range(width):
            #img[i][j][1]=0
            #img[i][j][2]=0
            #Umbral original = 128
            if(img[i][j][0]<150):
                img[i][j][0]=0
            else:
                img[i][j][0]=255

            if(img[i][j][1]<150):
                img[i][j][1]=0
            else:
                img[i][j][1]=255

            if(img[i][j][2]<150):
                img[i][j][2]=0
            else:
                img[i][j][2]=255

    
    # Guardar el frame procesado en el archivo de video de salida
    salida.writeFrame(img)
salida.close()


